In [1]:
import os

# Set the path to the image folder
folder = 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images'

# Get the list of all image files in the folder
files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.endswith('.jpg')]

In [2]:
import numpy as np

In [3]:
import pytesseract

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [4]:
files

['C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\430.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\431.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\432.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\433.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\434.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\435.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\436.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\437.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\438.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\439.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\440.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\441.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\442.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\443.jpg',
 'C:/Users/AMAR/OneDrive/Desktop/modeltext/val/images\\444.jpg',
 'C:/Users/AMAR/OneDrive/

In [5]:
for file in files:
    name=os.path.basename(file)
    print(name)

430.jpg
431.jpg
432.jpg
433.jpg
434.jpg
435.jpg
436.jpg
437.jpg
438.jpg
439.jpg
440.jpg
441.jpg
442.jpg
443.jpg
444.jpg
445.jpg
446.jpg
447.jpg
448.jpg
449.jpg
450.jpg
451.jpg
452.jpg
453.jpg
454.jpg
455.jpg
456.jpg
457.jpg
458.jpg
459.jpg
460.jpg
461.jpg
462.jpg
463.jpg
464.jpg
465.jpg
466.jpg
467.jpg
468.jpg
469.jpg
470.jpg
471.jpg
472.jpg
473.jpg
474.jpg
475.jpg
476.jpg
477.jpg
478.jpg
479.jpg
480.jpg
481.jpg
482.jpg
483.jpg
484.jpg
485.jpg
486.jpg
487.jpg
488.jpg
489.jpg
490.jpg
491.jpg
492.jpg
493.jpg
494.jpg
495.jpg
496.jpg
497.jpg
498.jpg
499.jpg


In [6]:
import cv2
import numpy as np
import os
import pytesseract as pt

In [7]:
# settings
INPUT_WIDTH =  640
INPUT_HEIGHT = 640

In [8]:
# LOAD THE IMAGE
#img = cv2.imread('12.jpg')

# LOAD YOLO MODEL
net = cv2.dnn.readNetFromONNX('textlabelweights.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [9]:
def get_detections(img,net):
    # CONVERT IMAGE TO YOLO FORMAT
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    # GET PREDICTION FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]
    
    return input_image, detections

def non_maximum_supression(input_image,detections):
    # FILTER DETECTIONS BASED ON CONFIDENCE AND PROBABILIY SCORE
    # center x, center y, w , h, conf, proba
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detecting license plate
        if confidence > 0.4:
            class_score = row[5] # probability score of license plate
            if class_score > 0.25:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    # clean
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
    # NMS
    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()
    
    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    # drawings
    for ind in index:
        x,y,w,h =  boxes_np[ind]
        bb_conf = confidences_np[ind]
        #conf_text = 'textlabel: {:.0f}%'.format(bb_conf*100)
        label_text = extract_text(image,boxes_np[ind])
        


        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+30),(0,0,0),-1)


        #cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,label_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)

    return image,label_text

In [10]:
# predictions
results=[]
def yolo_predictions(img,net):

    ## step-1: detections
        input_image, detections = get_detections(img,net)
    ## step-2: NMS
        boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)
    ## step-3: Drawings
        result_img,label_text = drawings(img,boxes_np,confidences_np,index)
        results.append({'Image Name': name, 'Text': label_text})
        #print(label_text)
        return results

In [11]:
def extract_text(image,bbox):
    x,y,w,h = bbox
    roi = image[y:y+h, x:x+w]
    
    if 0 in roi.shape:
        return ''
    
    else:
        text = pytesseract.image_to_string(roi)
        text = text.strip()
        
        return text

In [12]:
for file in files:
        img = cv2.imread(file)
        name=os.path.basename(file)
        predicted_results = yolo_predictions(img,net)

In [13]:
#cv2.namedWindow('results',cv2.WINDOW_KEEPRATIO)
#cv2.imshow('results',results)
#cv2.waitKey()
#cv2.destroyAllWindows()

In [14]:
predicted_results

[{'Image Name': '430.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '431.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '432.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '433.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '434.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '435.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '436.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '437.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '438.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '439.jpg', 'Text': '‘Model : 48962'},
 {'Image Name': '440.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '441.jpg', 'Text': '‘Model : 48962'},
 {'Image Name': '442.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '443.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '444.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '445.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '446.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '447.jpg', 'Text': 'Model : 48962'},
 {'Image Name': '448.jpg', 'Text': 'Model : 

In [15]:
import pandas as pd
df = pd.DataFrame(predicted_results)

# Save the DataFrame to an Excel file
df.to_excel('output.xlsx', index=False)